<a href="https://colab.research.google.com/github/PedroTechy/CarrisInsight/blob/streaming_development/spark_jobs/extract_carris_vehicles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 1: Authenticate with Google Cloud


In [1]:
from google.colab import auth
auth.authenticate_user()

# Step 2: Install Spark and BigQuery connector

In [22]:
# Install OpenJDK 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download Spark from a reliable source
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz

# Extract the downloaded Spark tarball
!tar xf spark-3.2.1-bin-hadoop3.2.tgz

!rm -rf spark-3.2.1-bin-hadoop3.2.tgz # No longer needed

# Install the GCS Connector
!wget -q https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar -P /content/spark-3.2.1-bin-hadoop3.2/jars/

# Install Findspark
!pip install -q findspark


# Step 3: Set environment variables for Spark and Java

In [23]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

# Step 4: Install BigQuery Connector for Spark

In [24]:
!wget -q https://storage.googleapis.com/spark-lib/bigquery/spark-bigquery-with-dependencies_2.12-0.29.0.jar


# Step 5: Initialize Spark session

In [25]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("GCS to BigQuery Streaming") \
    .config("spark.jars", "/content/spark-3.2.1-bin-hadoop3.2/jars/gcs-connector-hadoop3-latest.jar") \
    .getOrCreate()

# Step 6: Define GCS input path and output BigQuery table

In [26]:
input_path = "gs://edit-de-project-streaming-data/carris-vehicles"
output_table = "data-eng-dev-437916.data_eng_project_group3.raw_vehicles"

# Step 7: Read streaming data from GCS

In [27]:
# Configure Hadoop for GCS
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("google.cloud.auth.service.account.enable", "false")

streaming_df = spark.readStream \
    .format("json") \
    .load(input_path)

Py4JJavaError: An error occurred while calling o44.load.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2667)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.execution.datasources.DataSource.sourceSchema(DataSource.scala:249)
	at org.apache.spark.sql.execution.datasources.DataSource.sourceInfo$lzycompute(DataSource.scala:118)
	at org.apache.spark.sql.execution.datasources.DataSour

# Step 8: Write streaming data to BigQuery with auto-table creation

In [ ]:
streaming_query = streaming_df.writeStream \
    .format("bigquery") \
    .option("table", output_table) \
    .option("checkpointLocation", "gs://edit-data-eng-project-group3/streaming_data/checkpoints") \
    .option("temporaryGcsBucket", "gs://edit-data-eng-project-group3/streaming_data/temp-bucket") \
    .option("writeDisposition", "WRITE_APPEND") \
    .option("createDisposition", "CREATE_IF_NEEDED") \
    .outputMode("append") \
    .start()

# Step 9: Wait for the streaming query to finish

In [ ]:
streaming_query.awaitTermination()